# Some Notes on Performance for Python Code

<figure class="image" style="width:30%">
  <img src="https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/_images/faster.jpeg" alt="Picard, with caption 'Make it faster, Number One.">
    <figcaption><i>What we're doing today.</i></figcaption>
</figure>

In this one-off lecture, we'll consider a question that often pops up when writing Python code: 

> How can I make my code *faster*? 

There are a large number of things to keep in mind when considering how to make our code performant. Today, we'll briefly survey just a few of the possibilities. Here's a [nice list](https://stackify.com/20-simple-python-performance-tuning-tips/) of useful tips. 

## Numpy and Pandas

As you know, code that is primarily related to numerical computations or manipulation of rectangular data should almost always be performed in Numpy or Pandas. Both of these tools bring considerable performance improvements over base Python code in the situations to which they are suited, and should generally be used when possible. I won't be talking more about these topics today, but please remember that using these tools for appropriate problems is one of the easiest and biggest wins for performance. 

## 0. Should I Write This Code? 

So, you've got a task that you'd like to perform in Python, and you can see the outlines of a good code solution. *Should you write code for your solution?* Maybe! Before you do, take a moment to look around and see what else might be available. Public and well-established functions will almost always be more reliable and performant than what you hack together. Use them whenever possible! 

Suppose we'd like to iterate through all possible pairs of elements from a list. Here's one way: 

In [1]:
captains = ["Picard", "Sisko", "Janeway", "Archer", "Georgiou"]



[('Picard', 'Picard'),
 ('Picard', 'Sisko'),
 ('Picard', 'Janeway'),
 ('Picard', 'Archer'),
 ('Picard', 'Georgiou'),
 ('Sisko', 'Picard'),
 ('Sisko', 'Sisko'),
 ('Sisko', 'Janeway'),
 ('Sisko', 'Archer'),
 ('Sisko', 'Georgiou'),
 ('Janeway', 'Picard'),
 ('Janeway', 'Sisko'),
 ('Janeway', 'Janeway'),
 ('Janeway', 'Archer'),
 ('Janeway', 'Georgiou'),
 ('Archer', 'Picard'),
 ('Archer', 'Sisko'),
 ('Archer', 'Janeway'),
 ('Archer', 'Archer'),
 ('Archer', 'Georgiou'),
 ('Georgiou', 'Picard'),
 ('Georgiou', 'Sisko'),
 ('Georgiou', 'Janeway'),
 ('Georgiou', 'Archer'),
 ('Georgiou', 'Georgiou')]

The slowest run took 4.49 times longer than the fastest. This could mean that an intermediate result is being cached.
8.73 µs ± 4.46 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


This code looks pretty fast, but a quick Google search for "iterate through all pairs python" turns up this [StackOverflow](https://stackoverflow.com/questions/942543/operation-on-every-pair-of-element-in-a-list/37907649) post, indicating that I should use `itertools.product()`. 

1.72 µs ± 99 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Use of `itertools.product()` saved me from writing some code, and is also noticeably faster. Both kinds of savings will tend to become more pronounced the more complex the task that you need to accomplish. 

## 1. *Should* I Optimize This Code? 

Let's now suppose that you've done your research and determined that you do need to write up your own solution. Indeed, you've already done it. Great job! But now we face the next question: should you spend time making your code faster? 

<figure class="image" style="width:30%">
  <img src="https://media.makeameme.org/created/one-does-not-d06ab06f16.jpg" alt="Boromir, with caption 'One does not simply make some code run fast.">
    <figcaption><i></i></figcaption>
</figure>

When creating any kind of code product, we face various decisions about how to best allocate our time and energy. Making code faster can be a very helpful or even necessary thing to do. Before choosing to spend effort optimizing your code, ask yourself the following questions: 

1. Does my code run in practical (non-annoying) time on realistic cases? 
2. How many times will my code be run? By how many people? 
3. Is the code that I am considering optimizing a major part of my overall project? 

This third question is especially important. Maybe you have a function that takes 1 ms to run. This might be considered relatively slow! Maybe you can improve it to 0.1 ms with effort, a 10x speed up. But if that function will only be called once or twice in a given interaction, your user is unlikely to notice the difference. A function that takes 1 µs but which is called 1M times is much more important to optimize, and even a 2x speedup will have a much larger benefit for your user. 

## 2. *Which* Code Should I Optimize? 

Suppose you have a function or even an entire program. Which parts of it need your attention? A *profiler* can be used to analyze your code and provide important information, like: 

1. How many times are various functions within your code called? 
2. How much time does each function take per call? 

There are a number of different profilers available for Python. `cProfile` comes bundled with all modern Python distributions and is a traditional choice. We'll illustrate cProfile here. Another option I found is [Yappi](https://pypi.org/project/yappi/) (Yet Another Python ProfIler), which has several advanced features. It can be installed via the Anaconda Environments Pane. 

Let's write a simple function to count distinct elements in a list. 

Here's some data on which we'll call our function. 

In [7]:
s = """
Space, the final frontier
These are the voyages of the Starship Enterprise
Its five year mission
To explore strange new worlds
To seek out new life
And new civilizations
To boldly go where no one has gone before"""

L = (s*100).lower().split()
len(L), len(set(L))

(3800, 32)

{'space,': 100,
 'the': 300,
 'final': 100,
 'frontier': 100,
 'these': 100,
 'are': 100,
 'voyages': 100,
 'of': 100,
 'starship': 100,
 'enterprise': 100,
 'its': 100,
 'five': 100,
 'year': 100,
 'mission': 100,
 'to': 300,
 'explore': 100,
 'strange': 100,
 'new': 300,
 'worlds': 100,
 'seek': 100,
 'out': 100,
 'life': 100,
 'and': 100,
 'civilizations': 100,
 'boldly': 100,
 'go': 100,
 'where': 100,
 'no': 100,
 'one': 100,
 'has': 100,
 'gone': 100,
 'before': 100}

296 ms ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Hmmm, that's pretty slow. Was it the way that we're updating our dictionary, maybe? Should we seek a faster method than `count()` for actually counting objects? For these kinds of questions, profiling is exactly what we need: 

         7604 function calls in 0.349 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     3800    0.333    0.000    0.333    0.000 {method 'count' of 'list' objects}
        1    0.013    0.013    0.349    0.349 <ipython-input-6-80eaeb576a90>:1(count_list)
     3800    0.003    0.000    0.003    0.000 {method 'update' of 'dict' objects}
        1    0.000    0.000    0.349    0.349 {built-in method builtins.exec}
        1    0.000    0.000    0.349    0.349 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




The cProfile output contains lines for each of the function calls that the profiler was able to detect (note: it doesn't always detect all functions used). Note that dictionary update takes essentially no time at all, while the `.count()` method occupies the bulk of the time spent executing this function. To reduce this, we essentially have three options: 

1. We can reduce the time spent executing each `count()` call, by using a faster method. 
2. We can reduce the number of times `count()` is executed. 
3. We can completely refactor our code. 

I don't know of a faster version of `count()`, but it's not hard to dramatically reduce the number of times we execute it: 

2.4 ms ± 66.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


That's **much** faster! cProfile can tell us why: 

         68 function calls in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.004    0.004 <ipython-input-11-85a884abd86e>:1(count_list2)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        1    0.000    0.000    0.004    0.004 {built-in method builtins.exec}
       32    0.003    0.000    0.003    0.000 {method 'count' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       32    0.000    0.000    0.000    0.000 {method 'update' of 'dict' objects}




We dramatically reduced the number of times that the `count()` function was called, resulting in a large speed-up. 

In fact, we can do even better than this by changing our entire strategy. Instead of `count`ing each distinct element, we'll instead simply initialize a count and add to it each time we encounter the appropriate item. This works well because dictionary operations are extremely fast: 

821 µs ± 61.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


         3804 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 <ipython-input-14-194da5bdcb90>:1(count_list3)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
     3800    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}




cProfile tells us that we have a large number of calls to `dict.get()`, but access of dictionary elements is exceptionally fast (that's what dictionaries are optimized for), so this doesn't pose a major problem. 

However, we could do even better still by going back to Question 0: *Should* I write this code?

295 µs ± 20.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


         12 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-17-a8d4cf1f8ffb>:3(count_list4)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:548(__init__)
        1    0.000    0.000    0.000    0.000 __init__.py:617(update)
        1    0.000    0.000    0.000    0.000 abc.py:137(__instancecheck__)
        1    0.000    0.000    0.000    0.000 {built-in method _abc._abc_instancecheck}
        1    0.000    0.000    0.000    0.000 {built-in method _collections._count_elements}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method '

Relative to our initial version of the code, we've achieved a 1000x speedup by relying on built-in functions. Even our relatively efficient hand-written version is roughly 4x slower. 

# Various Tips

## Reduce Function Calls

The simple act of calling a function in Python can carry some performance overhead. For this reason, reducing the number of times that functions are called can lead to performance improvements, even when there's no otherwise-obvious redundancy. 

'before gone has one no where go boldly To civilizations new And life new out seek To worlds new strange explore To mission year five Its Enterprise Starship the of voyages the are These frontier final the Space,'

This function is logically coded, but it involves many calls to the `__add__` method of strings. Unfortunately, detecting this issue can be difficult for cProfile: 

17.9 µs ± 678 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


         6 function calls in 0.044 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.036    0.036    0.042    0.042 <ipython-input-20-a9e66526ba01>:1(reverse1)
        1    0.002    0.002    0.044    0.044 <string>:1(<module>)
        1    0.000    0.000    0.044    0.044 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.006    0.006    0.006    0.006 {method 'split' of 'str' objects}




The output doesn't make clear that there are any issues involving functions being called multiple times. But if we know to look out for this issue, then we might decide to use other tools, with the benefit of shortening our code. 

4.31 µs ± 55.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


         6 function calls in 0.102 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.024    0.024    0.102    0.102 <ipython-input-24-575c4362d720>:1(reverse2)
        1    0.000    0.000    0.102    0.102 <string>:1(<module>)
        1    0.000    0.000    0.102    0.102 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.074    0.074    0.074    0.074 {method 'join' of 'str' objects}
        1    0.003    0.003    0.003    0.003 {method 'split' of 'str' objects}




## Multithreading

A popular myth from a few years back asserted that most humans use only 10% of their brain most of the time. This is [not in fact true](https://en.wikipedia.org/wiki/Ten_percent_of_the_brain_myth). What **is** true is that your Python process uses only (about) 10% of the full computational resources of your computer at any given moment. 

More concretely, most modern personal (Intel-based) computers  possess somewhere between 2 and 6 *cores*, with 4 being perhaps the most common number. Each of these cores usually possesses 2 *threads*. Each thread is a separate process that can be used for executing tasks. When we run Python, we typically do so in a single-threaded environment, which means that our commands are executed by only one of our computer's threads. If your machine has 4 cores, and therefore 8 threads, this corresponds to only 12.5% of your overall CPU resources. 

*Multithreading* allows us to perform computations using more than one core. For large jobs, this can be beneficial. If your problem admits multithreading, if you use 4 threads, you may be able to finish your computation roughly 4 times as quickly. Nice! 

### Caution

Multithreading sounds exciting, and indeed, it can be a useful thing to do. However, multithreading should usually be your **last** measure for improving performance. Here's why: 

1. The speedup offered by multithreading is limited by the number of threads available on your computer. In my case, for example, my laptop has 8 threads. This means that the largest possible benefit I could ever obtain from multithreading is an 8x speed up. That's not nothing, but one can often obtain better speedups by revising one's logic and coding constructs, as demonstrated above. 
2. Multithreading can also place undue burdens on your hardware. There's **a reason** that many processes default to single-threaded runtimes. Your laptop simply isn't built to have all of its computational power engaged all the time. Using many threads for extended periods will likely overheat your system, resulting in thermal throttling. Your CPU will automatically reduce the rate of computation on each thread in order to dissipate heat. Thus, you could actually end up with **slower** code this way. 
3. There's overhead associated with distributing jobs between threads and syncing them back up. This overhead is manifest both in the time you need to spend writing code in order to manage these processes, and in the actual computation time as well. 
4. Multithreading really only "works" for processes that can be decoupled into independent components or chunks of data. Complex processes in which the current results of computation depend on previous results are often not suitable for multithreading. 

For these reasons, multithreading should only be considered **after** you're pretty confident that you've exhausted your other means of speeding up your code. 

### Example

Let's finish up with a simple example of multithreading in action. This is a toy example rather than a practical illustration. In a coming Discussion, we will work through some of the important and occasionally subtle associated with successfully deploying multithreading in practice. 

The simplest way to spawn multiple threads is through the `threading` module. One first defines a function which should be executed on each thread. This function presumably performs some time-intensive computation. One then spawns the desired number of `threading.Thread()` objects with `target` equal to the function which should be executed. Finally, one then `start`s each thread, and calls `join()` to ensure that no code is executed in the main process before each of the spawned threads finishes their work. 

This example is lightly modified from one presented [here](https://www.geeksforgeeks.org/multithreading-python-set-1/).

In [27]:
import threading
import os

def task1():
    print("Task 1 assigned to thread: {}".format(threading.current_thread().name))
    print("ID of process running task 1: {}".format(os.getpid()))
    print("To boldly go")
    print()
    
def task2():
    print("Task 2 assigned to thread: {}".format(threading.current_thread().name))
    print("ID of process running task 2: {}".format(os.getpid()))
    print("Where no one has gone before")
    print()
  

ID of process running main program: 6724
Main thread name: MainThread

Task 1 assigned to thread: THREAD 1Task 2 assigned to thread: THREAD 2
ID of process running task 2: 6724
Where no one has gone before


ID of process running task 1: 6724
To boldly go

